# Object Following - Live Demo

このノートブックでは、JetBotで物体を追跡する方法を示します。 collision avoidanceをベースに、「free(直進する)」時に物体を追跡します。\
物体検出に使うモデルは一般的な90種類のオブジェクトの画像を分類した[COCOデータセット](http://cocodataset.org)を事前にトレーニングしたssd_mobilenet_v2モデルを利用します。\
このモデルはTensorRTに変換したものを使用しますが、JetPackバージョンによってTensorRTのバージョンが異なるため、変換時のTensorRTバージョンと同一の実行環境である必要があります。

追跡可能な物体はCOCOデータセットで学習している物体となります。

* 人（インデックス1）
* カップ（インデックス47）

その他多数あります（クラスインデックスの完全なリストについては、[ラベルファイル](https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt)で確認できます）。\
インデックス0はbackgroundになります。通常、分類・検出するモデルでは「未検出」という状態を持つためにbackgroundラベルが使われています。\
学習済みモデルは[Tensorflow Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection)で公開されているものをベースに予めTensorRT化してあるものを使います。``Tensorflow Object Detection API``を使って自前のデータをデスクトップPCやクラウドサーバーで学習することも出来ます。

ssd_mobilenet_v2_cocoをTensorRTに変換することにより、物体検出モデルの実行が非常に高速になり、Jetson Nanoでリアルタイムに実行できるようになります。ただし、このノートブックではCOCOデータセットからのトレーニングや他の最適化に関する手順は実行しません。また、TensorRTはバージョンによりAPIが頻繁に変更されているため、他のJetPackバージョンで動作していたssd_mobilenet_v2_coco.engineは利用できません。

まずは始めてみましょう。

## Create Live Camera (カメラの準備)
本サンプルを実行するにあたり、まずnvargus-daemon(カメラ等で使用)をリスタートします。

In [ ]:
!echo jetbot | sudo -S systemctl restart nvargus-daemon

次に、カメラを初期化しましょう。物体検出モデルは300x300ピクセルの画像を入力とするため、カメラ解像度を300x300に設定します。

> 内部的には、CameraクラスはGStreamerを使用してJetson Nanoのイメージシグナルプロセッサ（ISP）を利用しています。これはCPUでリサイズ処理を実行する場合とは比較にならないほど超高速です。

In [ ]:
from jetbot import Camera

camera = Camera.instance(width=300, height=300, fps=21)

事前トレーニング済みのSSDエンジンを使用する[ObjectDetector](https://github.com/NVIDIA-AI-IOT/jetbot/blob/master/jetbot/object_detection.py)クラスをインポートして、ssd_mobilenet_v2_coco.engineをロードします。

### SSD MobileNet V2モデルを読み込む

In [ ]:
from jetbot import ObjectDetector

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

内部的には、``ObjectDetector``クラスはTensorRT Python APIを使用してモデルを実行します。また、モデルへの入力の前処理や、検出されたオブジェクトの解析も行います。 現時点では、``jetbot.ssd_tensorrt``パッケージを使用して作成されたモデルでのみ機能します。このパッケージには、モデルをTensorflowオブジェクト検出APIから最適化されたTensorRTエンジンに変換するためのユーティリティが含まれています。

次に、カメラ入力を使用してネットワークを実行してみましょう。 デフォルトでは ``ObjectDetector``クラスはカメラが生成する``bgr8``フォーマットを期待しています。 しかし、別のフォーマットを入力に使う場合は、デフォルトの前処理関数をオーバーライドして変更できます。

In [ ]:
detections = model(camera.value)

print(detections)

カメラ画像にCOCOオブジェクトがある場合、その情報は``detections``変数に格納されています。

### テキスト領域に検出を表示する

次のコードを使用して、検出されたオブジェクトの情報をテキストエリアに表示します。

In [ ]:
from IPython.display import display
import ipywidgets.widgets as widgets

detections_widget = widgets.Textarea()

detections_widget.value = str(detections)

display(detections_widget)

カメラ画像で検出された各オブジェクトのラベルID、信頼度、境界ボックスの座標が表示されます。

ミニバッチ学習時に複数の画像を一度に学習したなごりで、予測時にも一度に複数の画像を入力として期待するモデルに仕上がっています。\
今回は1台のカメラしか使わないため、モデルの入力には1枚の画像を持つ配列が使われています。\
最初の画像で検出された最初のオブジェクトのみを表示するには、次のように呼び出すことができます。

> オブジェクトが検出されない場合、エラーになるため、try-exceptでエラーハンドリングします

In [ ]:
image_number = 0
object_number = 0

try:
    print(detections[image_number][object_number])
except:
    print("object not found")

### 中心物体を追跡するようにロボットを制御する

次に、ロボットに指定されたクラスのオブジェクトを追跡させます。 これを行うには、次のようにします

1.  指定したクラスに一致するオブジェクトを検出します。[ラベルファイル](https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt)でラベルIDと対応する物体を確認してください。
2.  カメラの視野の中心に最も近いオブジェクトを選択します。これが指定したオブジェクトの時に追跡するターゲットになります。
3.  ロボットをターゲットオブジェクトに向けます。
4.  collision avoidanceをベース動作にしているため、障害物によってブロックされていると判断した場合は、左折します。

> ラベルファイルにはいくつかバージョンがあります。Tensorflowのラベルは80オブジェクト分になります。\
そのため、いくつか名前のないラベルが含まれています。[cocoデータセットのラベルについて](https://tech.amikelive.com/node-718/what-object-categories-labels-are-in-coco-dataset/)

また、ターゲットオブジェクトのラベル、ロボットの速度を制御するために使用するいくつかのウィジェットを作成します。
`turn gain`は、ターゲットオブジェクトとロボットの視野の中心との間の距離に基づいてロボットが回転する速度を制御します。

まず、衝突回避モデルをロードします。
衝突回避の例に従って、実際の環境でうまく動作するモデルを使用することをお勧めします。

In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

collision_model = torchvision.models.alexnet(pretrained=False)
collision_model.classifier[6] = torch.nn.Linear(collision_model.classifier[6].in_features, 2)
collision_model.load_state_dict(torch.load('../collision_avoidance/best_model.pth'))
device = torch.device('cuda')
collision_model = collision_model.to(device)

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

それでは、ロボットを初期化して、モーターを制御できるようにしましょう。

In [ ]:
from jetbot import Robot

robot = Robot()

コントロールウィジェットとカメラ更新とモデル実行の関数を作成します。

In [ ]:
from jetbot import bgr8_to_jpeg

blocked_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.Dropdown(
    options=['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
             'fire hydrant', '12', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep',
             'cow', 'elephant', 'bear', 'zebra', 'giraffe', '26', 'backpack', 'umbrella', '29', '30',
             'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
             'skateboard', 'surfboard', 'tennis racket', 'bottle', '45', 'wine glass', 'cup', 'fork', 'knife', 'spoon',
             'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut',
             'cake', 'chair', 'couch', 'potted plant', 'bed', '66', 'dining table', '68', '69', 'toilet',
             '71', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
             'sink', 'refrigerator', '83', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'],
    value='person',
    description='tracked label',
    disabled=False
)

speed_widget = widgets.FloatSlider(value=0.0, min=0.0, max=1.0, description='speed')
turn_gain_widget = widgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='turn gain')

width = int(image_widget.width)
height = int(image_widget.height)

"""font settings"""
fontScale = height/1000.0
if fontScale < 0.4:
    fontScale = 0.4
fontThickness = 1 + int(fontScale)
fontFace = cv2.FONT_HERSHEY_SIMPLEX

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection
        
def execute(change):
    image = change['new']
    
    # execute collision model to determine if blocked
    collision_output = collision_model(preprocess(image)).detach().cpu()
    prob_blocked = float(F.softmax(collision_output.flatten(), dim=0)[0])
    blocked_widget.value = prob_blocked
    
    # turn left if blocked
    if prob_blocked > 1.1:
        robot.left(0.3)
        image_widget.value = bgr8_to_jpeg(image)
        return
        
    # compute all detected objects
    detections = model(image)
    
    # draw all detections on image
    display_str = []
    display_str.append("detection info")
    for det in detections[0]:
        if det['label']  == 0:
            # background. skip
            #continue
            pass
        if det['confidence'] <= 0.2:
            # bad score. skip
            #continue
            pass
        bbox = det['bbox']
        score = det['confidence']
        label = det['label']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
        """get text info"""
        display_str.append("label:{} score:{:.2f}".format(label_widget.options[int(label)+1], score))
        #cv2.putText(image, display_str, org=(10, 20+20*num_detection), fontFace=fontFace, fontScale=fontScale, thickness=fontThickness, color=(77, 255, 9))

    """draw detection info"""
    max_text_width = 0
    max_text_height = 0
    if len(display_str) > 0:
        [(text_width, text_height), baseLine] = cv2.getTextSize(text=display_str[0], fontFace=fontFace, fontScale=fontScale, thickness=fontThickness)
        x_left = int(baseLine)
        y_top = int(baseLine)
        for i in range(len(display_str)):
            [(text_width, text_height), baseLine] = cv2.getTextSize(text=display_str[i], fontFace=fontFace, fontScale=fontScale, thickness=fontThickness)
            if max_text_width < text_width:
                max_text_width = text_width
            if max_text_height < text_height:
                max_text_height = text_height
        for i in range(len(display_str)):
            cv2.putText(image, display_str[i], org=(x_left, y_top + int(max_text_height*1.2 + (max_text_height*1.2 * i))), fontFace=fontFace, fontScale=fontScale, thickness=fontThickness, color=(77, 255, 9))

    # select detections that match selected class label
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.index)+1]
    
    # get detection closest to center of field of view and draw it
    target = closest_detection(matching_detections)
    if target is not None:
        bbox = target['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)


    # otherwise go forward if no target detected
    if target is None:
        robot.forward(float(speed_widget.value))
        
    # otherwsie steer towards target
    else:
        # move robot forward and steer proportional target's x-distance from center
        center = detection_center(target)
        robot.set_motors(
            float(speed_widget.value + turn_gain_widget.value * center[0]),
            float(speed_widget.value - turn_gain_widget.value * center[0])
        )
    
    # update image widget
    image_widget.value = bgr8_to_jpeg(image)

``start jetbot``ボタンを押すことでJetBotが動作するようになります。

In [ ]:
import ipywidgets
import time

model_start_button = ipywidgets.Button(description='start jetbot')
model_stop_button = ipywidgets.Button(description='stop jetbot')

model_widget = ipywidgets.VBox([
    image_widget,
    ipywidgets.HBox([label_widget, blocked_widget]),
    ipywidgets.HBox([speed_widget, turn_gain_widget]),
    ipywidgets.HBox([model_start_button, model_stop_button])
])

display(model_widget)


def start_model(c):
    execute({'new': camera.value})
    camera.unobserve_all()
    camera.observe(execute, names='value')
model_start_button.on_click(start_model)
    
def stop_model(c):
    camera.unobserve(execute, names='value')
    #camera.unobserve_all()
    time.sleep(1)
    robot.stop()
model_stop_button.on_click(stop_model)

うごいた！\
ターゲットが検出されると緑色のボックスが表示され、ターゲット以外の検出された物体は青色のボックスで表示されます。\
衝突回避モデルによって「blocked(旋回する)」と判断された時、JetBotは左に曲がります。\
衝突回避モデルによって「free(直進する)」と判断された時、ターゲットを検出している場合はJetBotはターゲットを追跡するように動作します。\
衝突回避モデルによって「free(直進する)」と判断された時、ターゲットを検出していない場合は衝突回避モデルと同様に直進します。